# Pipelines y Automatización de Flujos de Trabajo

Bienvenido a esta nueva lección donde nos vamos a centrar en una de las herramientas más poderosas y eficientes que **Scikit-Learn** ofrece para la automatización de procesos en machine learning: los **pipelines**.

Vamos a aprender a construir y a utilizar unos recursos llamados **pipelines** que sirven para **facilitar tanto la implementación como la evaluación** de modelos.


### Pipelines en el mundo real

Primero lo vamos a definir por fueras del mundo de la programación, ya que esto nos va a ayudar a comprenderlo un poco mejor.

"**Pipeline**" en inglés significa **tuberías**. Ya sabes, una red de tubos que transportan algo como agua, gas o petróleo. Entonces un "**Pipeline**" es algo que canaliza un elemento fluido, para que se transporte desde un lugar a otro.

Este concepto se utiliza metafóricamente en diversas industrias para describir *procesos* que tienen una *secuencia lineal y continua*, donde la salida de una fase se convierte directamente en la entrada de la siguiente hasta llegar al final del proceso. Por ejemplo, en la industria manufacturera, un pipeline podría referirse al flujo de producción que describe el recorrido que hace la materia prima desde que ingresa a la industria, hasta el producto terminado.


### Pipelines en programación

En programación, o más específicamente en **Scikit-Learn**, un *pipeline* es una *secuencia de transformaciones que terminan de un modelo final*.

Lo que hace un pipeline es encapsular todas las secuencias de pasos que hemos visto, como el preprocesamiento y el modelado, para que puedan ser procesados como un todo. Esto nos asegura que todos los pasos se ejecuten en el orden correcto, de una forma que sea repetible, y que sean fáciles de ajustar y de validar.


### Creación de un Pipeline Básico

Para comprender cómo se implementa un pipeline en el código de Scikit-Learn, vamos a retomar un bloque de código que ya hemos usado. El código que ves en la siguiente celda, es el que usamos en la lección *"Introducción a Scikit-Learn"*, para identificar los *estimadores*, los *transformadores* y los *predictores*.

Es un código muy básico que hace lo siguiente:
+ toma la base de datos iris
+ divide sus datos en conjuntos de entrenamiento y prueba
+ escala los datos
+ transforma los datos
+ entrena al modelo
+ hace predicciones
+ evalua el desempeño de las predicciones

Quiero usar este código que ya conoces, para que veas cuál es la diferencia entre trabajar con y sin pipelines.

### Código original (sin pipelines)

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Cargamos el dataset
data = load_iris()
X = data.data
y = data.target

# Dividimos el dataset en conjunto de entrenamiento y de prueba
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.25, random_state=0)

# Creamos una instancia del escalador
scaler = StandardScaler()

# Estimador (StandardScaler): Aprendemos los parámetros de escalado con fit
scaler.fit(X_entrena)

# Transformador (StandardScaler): Aplicamos la transformación a los datos de entrenamiento y prueba
X_entrena_escalado = scaler.transform(X_entrena)
X_prueba_escalado = scaler.transform(X_prueba)

# Creamos una instancia del modelo
modelo = LogisticRegression()

# Estimador (LogisticRegression): Entrenamos el modelo con los datos escalados
modelo.fit(X_entrena_escalado, y_entrena)

# Predictor (LogisticRegression): Hacemos predicciones y evaluamos el modelo
y_pred = modelo.predict(X_prueba_escalado)
puntaje = modelo.score(X_prueba_escalado, y_prueba)
print(f"Las predicciones son: {y_pred}")
print(f"La precisión del modelo es: {puntaje:.2f}")

Las predicciones son: [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]
La precisión del modelo es: 0.97


### Código modificado (con Pipelines)

En la próxima celda he copiado ese mismo código, pero aquí he hecho las modificaciones necesarias para poner sus procesos dentro de un **pipeline**. Puedes ver ambas versiones, comparar las diferencias (que explicaré más abajo), y verificar que los resultados son exactamente iguales.

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Cargamos el dataset
data = load_iris()
X = data.data
y = data.target

# Dividimos el dataset en conjunto de entrenamiento y de prueba
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.25, random_state=0)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('modelo', LogisticRegression())
])

pipeline.fit(X_entrena, y_entrena)

# Predictor (LogisticRegression): Hacemos predicciones y evaluamos el modelo
y_pred = pipeline.predict(X_prueba)
puntaje = pipeline.score(X_prueba, y_prueba)

print(f"Las predicciones son: {y_pred}")
print(f"La precisión del modelo es: {puntaje:.2f}")

Las predicciones son: [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]
La precisión del modelo es: 0.97


### Implementación del pipeline

Ahora vamos a describir las diferencias entre ambos códigos, y a explicar por qué hemos hecho cada modificación.

#### 1. Importación de `Pipeline`

Lo primero que hicimos fue importar al recurso de Scikit-Learn que permite aplicar las pipelines
```
from sklearn.pipeline import Pipeline
```

##### 2. Declaración del pipeline

Lo segundo es declarar el pipeline en sí mismo. ¿cómo? modificando la forma en que declaramos al **escalador** y al **modelo**. En el *código original* lo hicimos en lineas separadas e independientes entre sí, de la siguiente manera:
```
scaler = StandardScaler()
modelo = LogisticRegression()
```
Pero en el código nuevo he eliminado esas líneas, y he declarado una variable `pipeline` que contiene la función `Pipeline()`, a la cual le pasamos como parámetro una *lista* compuesta de *tuplas* con los nombres y llamadas a los objetos en cuestión.
```
pipeline = Pipeline([
    ('scaler', StandardScaler()),   # Paso 1: Escalador
    ('modelo', LogisticRegression())  # Paso 2: Modelo
])
```

##### 3. Entrenamiento del pipeline

La ventaja de que ahora tengamos a estos dos elementos dentro de nuestra tubería, o **pipeline**, es que ya no vamos a necesitar entrenarlos por separado. Ahora solamente tengo que entrenar a mi pipeline, y eso aplica para cada uno de sus elementos internos.

Por lo tanto puedo eliminar las líneas de entrenamiento por separado:
```
scaler.fit(X_entrena)
modelo.fit(X_entrena_escalado, y_entrena)
```

Y directamente procedo a entrenal al pipeline.
```
pipeline.fit(X_entrena, y_entrena)
```

##### 4. Proceso de transformaciones dentro del piepline

La siguiente ventaja es que ahora no vamos a necesitar escribir el **proceso de transformación** que hacíamos con estas líneas dedicadas al escalamiento de los datos:
```
X_entrena_escalado = scaler.transform(X_entrena)
X_prueba_escalado = scaler.transform(X_prueba)
```

Esas lineas pueden ser eliminadas a partir de ahora ¿Por qué? Porque cuando usas un pipeline en Scikit-Learn, la necesidad de transformar explícitamente los datos de entrenamiento y prueba se elimina.

Esta es tal vez la parte más importante respecto de la magia de pipeline: el pipeline automáticamente gestiona de manera interna todas las transformaciones, y se asegura que se apliquen de manera adecuada, y en el orden correcto, durante el entrenamiento y la predicción. 
+ Durante el Entrenamiento (`fit()`): primero ajusta el `StandardScaler` a los datos de entrenamiento (`X_entrena`) para aprender los parámetros de escalado (es decir, la **media** y la **desviación estándar** de cada característica). Inmediatamente después, transforma esos mismos datos de entrenamiento usando los parámetros aprendidos. Después de escalar los datos de entrenamiento, el pipeline pasa estos datos transformados directamente al modelo `LogisticRegression`, que entonces se entrena con ellos.
+ Durante la Predicción (`predict()` y `score()`) el pipeline automáticamente transforma los datos de prueba (`X_prueba`) utilizando el mismo `StandardScaler` que fue ajustado a los datos de entrenamiento. Esto nos garantiza que los datos de prueba se escalen exactamente de la misma manera que los datos de entrenamiento.


##### 5. Predicción y evaluación del pipeline

Esto nos lleva a la última modificación que voy a hacer aquí, que es reemplazar `modelo` por `pipeline` en las predicciones y en la evaluación, y reemplazar estas variables que ya no existen, por las variables originales.
```
y_pred = pipeline.predict(X_prueba)
puntaje = pipeline.score(X_prueba, y_prueba)
```

Al ejecutarlo, como puedes ver, obtuve exactamente lo mismo que en el código sin pipelines, pero con un abordaje que no solo es más eficiente, sino que tiene otros beneficios clave:
- **Consistencia:** Garantiza que todos los datos se procesen de la misma forma. No hay riesgo de olvidar transformar los datos de prueba o de aplicar una transformación de manera incorrecta.
- **Simplicidad:** Simplifica el código y reduce la posibilidad de errores, especialmente en flujos de trabajo de machine learning más complejos.
- **Reproducibilidad:** Mejora la reproducibilidad de los resultados, ya que los mismos pasos y transformaciones se aplican de forma uniforme cada vez que se ejecuta el código.


### Pipelines aún más simplificados

Pero eso no es todo, hay una forma aún más simple de usar pipelines, que es a través de un recurso muy parecido, pero que se llama `make_pipeline`.

El siguiente código es una copia exacta del anterior (el que ya tiene `pipeline`), pero donde introduje una sutil modificación:

1. En lugar de importar `Pipeline`, he importado `make_pipeline`.
2. La variable `pipeline` ya no contiene a la función `Pipeline()`, sino a la función `make_pipeline`.
3. La función `make_pipeline()` no necesita que declaremos explícitamente los nombres de sus componentes, por lo que en lugar de una *lista* de *tuplas*, simplemente le pasamos los componentes en sí mismos.

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Cargamos el dataset
data = load_iris()
X = data.data
y = data.target

# Dividimos el dataset en conjunto de entrenamiento y de prueba
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.25, random_state=0)

pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression())

pipeline.fit(X_entrena, y_entrena)

# Predictor (LogisticRegression): Hacemos predicciones y evaluamos el modelo
y_pred = pipeline.predict(X_prueba)
puntaje = pipeline.score(X_prueba, y_prueba)

print(f"Las predicciones son: {y_pred}")
print(f"La precisión del modelo es: {puntaje:.2f}")

Las predicciones son: [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]
La precisión del modelo es: 0.97


Al ejecutarlo obtenemos los mismos resultados pero con un código aún más simplificado y robusto.

En esta lección, hemos explorado cómo los **pipelines** de **Scikit-Learn** nos permiten simplificar y automatizar el flujo de trabajo en proyectos de **Machine Learning**, desde el preprocesamiento de datos hasta la evaluación del modelo.

Al integrar estas herramientas en tu práctica, vas a poder hacer que tus proyectos sean más eficientes, más efectivos y más fáciles de manejar.